# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download and unzip the dataset into Colab's runtime environment

In [ ]:
!cp "/content/drive/My Drive/dataset.zip" "/content"

In [ ]:
!mkdir "/content/drive/My Drive/Colab Notebooks/Snapshot Ensemble"

In [ ]:
!zip -FF ds.zip --out ds2.zip

In [ ]:
!unzip -o -q "ds2.zip"

# Import Tensorflow and Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Data Directory

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
base_dir = '/content/dataset'

train_dir = os.path.join(base_dir, 'train')
test_dir = "/content/dataset/test" # Imbalanced test
test_dir_100 = "/content/drive/My Drive/Colab Notebooks/dataset/test_100" # Balanced test

# Initialization

In [ ]:
image_size = (240, 240) # efficienetB1

batch_size = 32

LEARNING_RATE=0.0001
num_classes=3
epochs = 30

# Training, Validation and Testing data load and Train Augmentation

In [ ]:
trdata=tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=90,
    shear_range=0.10,
    zoom_range=0.10,    
    validation_split=0.1
)

train_ds = trdata.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    class_mode="categorical",
    subset="training",
    batch_size=batch_size,
    shuffle=True,
)

valdata=tf.keras.preprocessing.image.ImageDataGenerator()

val_ds = trdata.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    class_mode="categorical",
    subset="validation",
    batch_size=batch_size,
)

testdata=tf.keras.preprocessing.image.ImageDataGenerator()

test_ds = testdata.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False,
)

testdata_100=tf.keras.preprocessing.image.ImageDataGenerator()

test_ds_100 = testdata_100.flow_from_directory(
    directory=test_dir_100,
    target_size=image_size,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False,
)

# Training, Validation and Testing data load and Train without Augmentation

In [ ]:
trdata=tf.keras.preprocessing.image.ImageDataGenerator( 
    validation_split=0.1   
)

train_ds = trdata.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    class_mode="categorical",
    subset="training",
    batch_size=batch_size,
    shuffle=True,
)

valdata=tf.keras.preprocessing.image.ImageDataGenerator()

val_ds = trdata.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    class_mode="categorical",
    subset="validation",
    batch_size=batch_size,
)

testdata=tf.keras.preprocessing.image.ImageDataGenerator()

test_ds = testdata.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False,
)

testdata_100=tf.keras.preprocessing.image.ImageDataGenerator()

test_ds_100 = testdata_100.flow_from_directory(
    directory=test_dir_100,
    target_size=image_size,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False,
)

# Class Weight

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_ds.classes), 
                train_ds.classes)

print(class_weights)
print(train_ds.class_indices)

# Efficientnet Installation

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

# Swish Activation

In [ ]:
from keras.backend import sigmoid
from keras.utils import get_custom_objects
from keras.layers import Activation

class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))


get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

# Cosine Annealing Snapshot Ensemble

In [ ]:
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor
 
# snapshot ensemble with custom learning rate schedule

class SnapshotEnsemble(Callback):
	# constructor
	def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
		self.epochs = n_epochs
		self.cycles = n_cycles
		self.lr_max = lrate_max
		self.lrates = list()

	# calculate learning rate for epoch
	def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
		epochs_per_cycle = floor(n_epochs/n_cycles)
		cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
		return lrate_max/2 * (cos(cos_inner) + 1)

	# calculate and set learning rate at the start of the epoch
	def on_epoch_begin(self, epoch, logs={}):
		# calculate learning rate
		lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
		# set learning rate
		backend.set_value(self.model.optimizer.lr, lr)
		# log value
		self.lrates.append(lr)

	# save models at the end of each cycle
	def on_epoch_end(self, epoch, logs={}):
		# check if we can save model
		epochs_per_cycle = floor(self.epochs / self.cycles)
	
		if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
			# save model to file
			#filename = "snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
			
			filename = "/content/drive/My Drive/Colab Notebooks/Snapshot Ensemble/snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
	
			self.model.save(filename)
			print('>saved snapshot %s, epoch %d' % (filename, epoch))


# Modified Model Build

In [ ]:
from keras import layers
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D,ZeroPadding2D
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.applications import DenseNet121
from keras.applications import MobileNetV2
from keras.regularizers import l2
from keras.regularizers import l1_l2

from keras import optimizers
import efficientnet.tfkeras as enet


base_model = enet.EfficientNetB1(include_top=False, input_shape=(240,240,3), pooling='avg', weights="imagenet",classes=num_classes)

x = base_model.output
x = BatchNormalization()(x)

x = Dense(512,kernel_regularizer=l1_l2(l1=1e-5, l2=1e-3))(x)
x = BatchNormalization()(x)
x = Activation(swish_act)(x)
x = Dropout(0.3)(x)

x = Dense(512,kernel_regularizer=l1_l2(l1=1e-5, l2=1e-3))(x)
x = BatchNormalization()(x)
x = Activation(swish_act)(x) 
x = Dropout(0.3)(x)

# Output layer
predictions = Dense(num_classes, activation="softmax")(x)

model= Model(inputs = base_model.input, outputs = predictions)

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=LEARNING_RATE),metrics=['acc'])
model.summary()
model.save('/content/drive/My Drive/Colab Notebooks/Snapshot Ensemble/model_base.h5') # Model save for ensemble

# Model Fit

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('training.log', separator=",", append=True)

checkpoint = ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/Snapshot Ensemble/model_weight.h5', monitor='val_acc',save_best_only=True,)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, verbose=1,)

n_cycles = epochs / 6
ca = SnapshotEnsemble(epochs, n_cycles, 0.0001)

csv_logger = CSVLogger("/content/drive/My Drive/Colab Notebooks/Snapshot Ensemble/training.log", separator=",", append=True)

history=model.fit(
    train_ds, epochs=epochs, callbacks=[ca,checkpoint,csv_logger], validation_data=val_ds
)